# Optimization

### Collect training features

In [11]:
import gc
import numpy as np
np.random.seed(0)
import random
random.seed(0)
import os, sys

directory = os.path.abspath('/Users/joris/Documents/Work/bsc ai/thesis/bachelor-thesis/code/')
sys.path.append(directory)

from data.loading import DataLoader
from lib.conceptors import *
from lib.esn import ESN
from lib.helpers import *
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical

import warnings
warnings.filterwarnings("ignore")

XorZ = "X"
#method = "centroids"
#method = "sims"
method = "pred"
#method = "ogsignals"

print("starting")

path = '../../data/'
fc = DataLoader(path)

dr = []
speakers = []

long_version = False
n_mels = 14
delta = False
delta_delta = False
subsamples = 10

path_option = "Final"+str(long_version)+str(n_mels)+str(delta)+str(delta_delta)+str(subsamples)

if dr:
    path_option = str(dr)+"_"+path_option
if len(speakers):
    path_option = str(speakers[0])+"_"+path_option

features,labels,_ = fc.collectFeaturesInSegments(
    n_mels=n_mels,delta=delta,delta_delta=delta_delta,
    long_version=long_version,speakers=speakers,dr=dr,
    subsamples=subsamples,path_option=path_option)

print(f"Loaded to {len(features)} samples of shape {features[0].shape}")

gc.collect()

starting
-from output
---- success
Loaded to 177080 samples of shape (10, 13)


541

### Regroup data and subset phonemes

In [12]:
def filter_data(features, labels, classes=None,trunkate=False,limit=1000000):
    group = {}

    for i in range(len(labels)):
        if classes != None and labels[i] not in classes:
            continue
        if labels[i] not in group.keys():
            group[labels[i]] = []
        if len(group[labels[i]]) < limit:
            group[labels[i]].append(features[i])

    samples_per_phoneme = min([ len(value) for value in group.values() ])
    if trunkate:
        for key in group.keys():
            group[key] = random.sample(group[key], samples_per_phoneme)

    classes = list(group.keys())
    filtered_labels = []
    filtered_features = []
    for label, features in group.items():
        for feature in features:
            filtered_features.append(feature)
            filtered_labels.append(label)
    return classes, filtered_features, filtered_labels

classes, features, labels = filter_data(features, labels, classes=None, trunkate=False, limit=1000000)

print(f"Filtered to {len(features)} samples of shape {features[0].shape}")

Filtered to 177080 samples of shape (10, 13)


### Compute conceptors

In [13]:
def compute_Cs_and_Ns(group, esn, aperture):
    Cs_clas = []
    for phoneme, signals in group.items():
        X = np.array([])
        for signal in signals:
            x = esn.run(signal.T, XorZ=XorZ)
            X = np.hstack((X, x)) if X.size else x
        Cs_clas.append(compute_c(X, aperture))
    print("optimizing")
    Cs_clas = optimize_apertures(Cs_clas)
    print("normalizing")
    Cs_clas = normalize_apertures(Cs_clas)
    print("- computing negative conceptors")
    Ns_clas = Ns_from_Cs(Cs_clas)
    #Ns_clas = optimize_apertures(Ns_clas)
    #Ns_clas = normalize_apertures(Ns_clas)
    return Cs_clas, Ns_clas

def compute_X_centroids(group, esn):
    centroids = []
    for phoneme, signals in group.items():
        reservoir_states = []
        for signal in signals:
            x = esn.run(signal.T, XorZ=XorZ)
            reservoir_states.append(x)
        reservoir_states = np.array(reservoir_states)
        centroids.append(np.mean(reservoir_states,axis=0))
    return centroids

def original_signal_centroids(group):
    centroids = []
    for phoneme, signals in group.items():
        signal_array = np.array(signals)
        centroids.append(np.mean(signal_array,axis=0))
    return centroids

def d(x,y):
    return np.linalg.norm(x-y)

# only optimize Cs before : g=133.3 a=0.5516
# optimize Cs and Ns after : g1=133.3 g2=.5 a=.
# only normalize Cs before : 
# normalize Cs and Ns after : 
# normalize Cs and Ns before and after : 
# similarities : a=0.2543922127255461

In [14]:
from sklearn.base import BaseEstimator, ClassifierMixin

class Classifier(BaseEstimator, ClassifierMixin):
    def __init__(self, W_in_scale=1.1, spectral_radius=2.57, b_scale=.44, weights=.1):
        self.W_in_scale = W_in_scale
        self.spectral_radius = spectral_radius
        self.b_scale = b_scale
        self.weights = weights

    def fit(self, X, y, **params):
        # Store the classes seen during fit
        self.X_ = X
        self.y_ = y
        
        # Group data by class
        group = {}
        for i in range(len(y)):
            if self.y_[i] not in group.keys():
                group[self.y_[i]] = []
            group[self.y_[i]].append(self.X_[i])
        self.classes = list(group.keys())
        samples_per_phoneme = min([ len(value) for value in group.values() ])
        self.n_samples = sum([len(x) for x in list(group.values())])
        print("Number of samples:", self.n_samples)
        print(self.classes)
        
        # Init Reservoir
        esn_params = {
            "in_dim": params["in_dim"],
            "out_dim": params["out_dim"],
            "N": 100,
            "W_in_scale": self.W_in_scale,
            "b_scale": self.b_scale,
            "spectral_radius": self.spectral_radius,
            "weights": self.weights
        }

        self.esn = ESN(esn_params)
        if method == "pred" or method == "sims":
            self.Cs_clas, self.Ns_clas = compute_Cs_and_Ns(group, esn=self.esn, aperture=1)
        elif method == "centroids":
            self.centroids = compute_X_centroids(group, esn=self.esn)
        else:
            self.centroids = original_signal_centroids(group)
        # Return the classifier
        return self

    def predict(self, X):
        y = []
        for sample in X:
            if method == "sims":
                x = self.esn.run(sample.T, XorZ=XorZ)
                C_loc = compute_c(x,aperture=133.3)
                es = [ similarity_c(C,C_loc) for C in self.Cs_clas ]
                es = [ np.sum(p) for p in es ]
                y.append(self.classes[np.argmax(es)])
            elif method == "centroids":
                x = self.esn.run(sample.T, XorZ=XorZ)
                es = [ d(x,centroid) for centroid in self.centroids ]
                y.append(self.classes[np.argmin(es)])
            elif method == "ogsignals":
                es = [ d(sample,centroid) for centroid in self.centroids ]
                y.append(self.classes[np.argmin(es)])
            else:
                x = self.esn.run(sample.T, XorZ=XorZ)
                es = evidences_for_Cs(x,self.Cs_clas,self.Ns_clas)
                if XorZ == "X":
                    es = [ np.sum(p) for p in es ]
                y.append(self.classes[np.argmax(es)])
                
        return y

In [ ]:
parameters = {
    'W_in_scale' : Real(0.05,2),
    'spectral_radius' : Real(0.05,4),
    'b_scale' : Real(0,2),
    'weights' : Real(0.05,0.95)
}

#opt = Classifier()

#opt = Classifier(1.5,1.5,.2,.1) # Z optimal params
opt = Classifier(1.1,2.57,.44,.1) # X optimal params
#opt = BayesSearchCV(Classifier(), parameters, n_iter=50, cv=3)
opt.fit(features, labels, **{
    "in_dim":n_mels,
    "out_dim":n_mels
})

Number of samples: 177080
['h#', 'w', 'ih', 's', 'ah', 'ch', 'n', 'ae', 't', 'v', 'r', 'f', 'y', 'uw', 'sh', 'l', 'b', 'iy', 'aa', 'd', 'eh', 'p', 'z', 'ey', 'dx', 'ay', 'ng', 'k', 'dh', 'er', 'm', 'jh', 'g', 'ow', 'aw', 'hh', 'uh', 'oy', 'th']


## Testing
### Feature Collection

In [ ]:
print("Testing...")

fv, lv, _ = fc.collectFeaturesInSegments(
    ft='Test',n_mels=n_mels,delta=delta,delta_delta=delta_delta,
    long_version=long_version,speakers=[],dr=dr,sentence=[],
    subsamples=subsamples,path_option=path_option+"_test")

print(len(fv))
classes, fv, lv = filter_data(fv, lv, classes=None,trunkate=True,limit=1000000)

print(opt.score(fv, lv))
#print(opt.best_params_)

#with open('res.txt','x') as file:
#    file.write(opt.score(fv,lv))
#    file.write(opt.best_params_)

# Disjunction method

In [40]:
def compute_Cs_and_Ns_below_phoneme(features, labels, esn, aperture):
    all_Cs = []
    Cs = {}
    for signal, label in zip(features, labels):
        x = esn.run(signal.T)
        C = compute_c(x, aperture)
        if label not in Cs.keys():
            Cs[label] = C
        else:
            Cs[label] = OR_C(Cs[label], C)
    #phonemes = list(Cs.keys())
    Cs = list(Cs.values())
    print("- optimizing +")
    Cs = optimize_apertures(Cs)
    Cs = normalize_apertures(Cs)
    print("- computing negative conceptors")
    Ns = Ns_from_Cs(Cs)
    return Cs, Ns